In [1]:
import matplotlib 
import numpy as np
import pylab as plt
import os
%matplotlib inline

In [2]:
def read_waveforms_from_file(filepath, start_line):
    data = np.loadtxt(filepath, delimiter=',', skiprows=start_line-1, usecols=(0,1))

    waveforms = []
    prev_time = data[0,0]
    current_time_list = []
    current_val_list = []

    for t, v in data:
        if t < prev_time and current_time_list:
            waveforms.append((np.array(current_time_list), np.array(current_val_list)))
            current_time_list = []
            current_val_list = []

        current_time_list.append(t)
        current_val_list.append(v)
        prev_time = t

    if current_time_list:
        waveforms.append((np.array(current_time_list), np.array(current_val_list)))

    return waveforms

In [3]:
start_line = 4505
base_path = '/Users/tikitorch/Downloads/R7378A_06262025'


folder_waveforms = {}
for folder_name in sorted(os.listdir(base_path)):
    folder_path = os.path.join(base_path, folder_name)
    if not (os.path.isdir(folder_path) and folder_name.startswith('900V_210p13')):
        continue

    all_waveforms_in_folder = []
    
    for filename in sorted(os.listdir(folder_path)):
        if not filename.startswith('C2Trace'):
            continue
        
        filepath = os.path.join(folder_path, filename)
        waveforms = read_waveforms_from_file(filepath, start_line)
        all_waveforms_in_folder.extend(waveforms)
    folder_waveforms[folder_name] = all_waveforms_in_folder

FileNotFoundError: [Errno 2] No such file or directory: '/Users/tikitorch/Downloads/R7378A_06262025'

In [ ]:
cutoff = 1e7

filtered_waveforms = {}

for folder_name, waves in folder_waveforms.items():
    filtered_waveforms[folder_name] = [] 
    
    for time_vals, amplitude_vals in waves:
        time_vals = np.array(time_vals)
        amplitude_vals = np.array(amplitude_vals)

        dt = np.mean(np.diff(time_vals))
        fft_vals = np.fft.fft(amplitude_vals)
        freqs = np.fft.fftfreq(len(amplitude_vals), dt)

        filter_mask = np.abs(freqs) < cutoff
        fft_filtered = fft_vals * filter_mask

        amplitude_filtered = np.fft.ifft(fft_filtered).real
        
        filtered_waveforms[folder_name].append((time_vals, amplitude_filtered))


In [4]:
plt.figure(figsize=(10, 6), dpi=150)

colors = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'cyan'] 
for i, (folder_name, waveforms) in enumerate(filtered_waveforms.items()):
    results = []
    
    for time_arr, val_arr in waveforms:
        min_val = np.min(val_arr)   # minimum amplitude value of filtered waveform
        results.append(min_val)
        
    plt.hist(results, bins=50, alpha=0.5, density=True, color=colors[i % len(colors)],
             label=folder_name, edgecolor='black')

plt.xlabel('Minimum FFT Magnitude')
plt.ylabel('Probability Density')
plt.title('Normalized Histogram of Minimum Magnitude Values - All Folders')
plt.legend(fontsize='small')
plt.grid(True)
plt.show()


NameError: name 'filtered_waveforms' is not defined

<Figure size 1500x900 with 0 Axes>

In [5]:
colors = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'cyan']

plt.figure(figsize=(10, 6), dpi=150)

for i, (folder_name, waveforms) in enumerate(filtered_waveforms.items()):
    times_of_arrival = []

    for time_vals, amplitude_vals in waveforms:
        time_vals = np.array(time_vals) - time_vals[0]
        amplitude_vals = np.array(amplitude_vals)

        V_min = np.min(amplitude_vals)
        min_idx = np.argmin(amplitude_vals)
        threshold = V_min / 2

        crossing_idx = None
        for idx in range(min_idx - 1, 0, -1):
            if amplitude_vals[idx] > threshold and amplitude_vals[idx + 1] <= threshold:
                crossing_idx = idx
                break

        if crossing_idx is not None:
            t1, v1 = time_vals[crossing_idx], amplitude_vals[crossing_idx]
            t2, v2 = time_vals[crossing_idx + 1], amplitude_vals[crossing_idx + 1]
            slope = (v2 - v1) / (t2 - t1)
            t_cross = t1 + (threshold - v1) / slope
            times_of_arrival.append(t_cross)
        else:
            times_of_arrival.append(np.nan)

    times_of_arrival = np.array(times_of_arrival)
    valid_times = times_of_arrival[~np.isnan(times_of_arrival)]

    if len(valid_times) == 0:
        print(f"No valid times for folder {folder_name}")
        continue
    plt.hist(valid_times, bins=50, alpha=0.5, density=True,
             color=colors[i % len(colors)],
             label=folder_name, edgecolor='black')

plt.xlabel('Time of Arrival (s)')
plt.ylabel('Normalized Counts (Probability Density)')
plt.title('Normalized Time of Arrival Histogram for All Folders')
plt.legend(fontsize='small')
plt.grid(True)
plt.show()

NameError: name 'filtered_waveforms' is not defined

<Figure size 1500x900 with 0 Axes>

In [6]:
colors = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'cyan']
zero_threshold = 1e-4  # threshold to treat as zero crossing, adjust if needed

plt.figure(figsize=(10, 6), dpi=150)

for i, (folder_name, waveforms) in enumerate(filtered_waveforms.items()):
    pulse_areas = []

    for time_vals, amplitude_vals in waveforms:
        time_vals = np.array(time_vals) - time_vals[0]  # normalize time start to zero
        amplitude_vals = np.array(amplitude_vals)

        # Find index of minimum amplitude (pulse minimum)
        min_idx = np.argmin(amplitude_vals)

        # Find zero crossing before min_idx (leading edge)
        leading_zero_idx = None
        for j in range(min_idx, 0, -1):
            if amplitude_vals[j] * amplitude_vals[j - 1] < 0 or abs(amplitude_vals[j]) < zero_threshold:
                leading_zero_idx = j
                break

        # Find zero crossing after min_idx (trailing edge)
        trailing_zero_idx = None
        for j in range(min_idx, len(amplitude_vals) - 1):
            if amplitude_vals[j] * amplitude_vals[j + 1] < 0 or abs(amplitude_vals[j]) < zero_threshold:
                trailing_zero_idx = j
                break

        # If both zero crossings found, integrate pulse area
        if leading_zero_idx is not None and trailing_zero_idx is not None and trailing_zero_idx > leading_zero_idx:
            idx_range = slice(leading_zero_idx, trailing_zero_idx + 1)
            area = np.trapz(amplitude_vals[idx_range], time_vals[idx_range])
            pulse_areas.append(area)
        else:
            # If zero crossings not found, skip this waveform or handle differently
            continue

    if len(pulse_areas) == 0:
        print(f"No valid pulse areas in folder {folder_name}")
        continue

    # Plot normalized histogram for this folder
    plt.hist(pulse_areas, bins=50, alpha=0.5, density=True,
             color=colors[i % len(colors)],
             label=folder_name, edgecolor='black')

plt.xlabel('Pulse Area (Integral)')
plt.ylabel('Normalized Counts (Probability Density)')
plt.title('Normalized Pulse Area Distribution for All Folders')
plt.legend(fontsize='small')
plt.grid(True)
plt.show()


NameError: name 'filtered_waveforms' is not defined

<Figure size 1500x900 with 0 Axes>

In [7]:
colors = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'cyan']

plt.figure(figsize=(12, 7), dpi=150)

for i, (folder_name, waveforms) in enumerate(filtered_waveforms.items()):
    if len(waveforms) == 0:
        continue

    min_time = min(w[0][0] for w in waveforms)
    max_time = max(w[0][-1] for w in waveforms)

    common_time = np.linspace(min_time, max_time, 1000)

    interpolated_amps = []
    for time_vals, amp_vals in waveforms:
        time_vals = np.array(time_vals)
        amp_vals = np.array(amp_vals)
        interp_amp = np.interp(common_time, time_vals, amp_vals)
        interpolated_amps.append(interp_amp)

    avg_amp = np.mean(interpolated_amps, axis=0)

    plt.plot(common_time, avg_amp, color=colors[i % len(colors)], label=folder_name)

plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.title('Average Waveforms per Folder')
plt.grid(True)
plt.legend(fontsize='small')
plt.show()


NameError: name 'filtered_waveforms' is not defined

<Figure size 1800x1050 with 0 Axes>

In [ ]:
cutoff = 2e7

filtered_waveforms = {}

for folder_name, waves in folder_waveforms.items():
    filtered_waveforms[folder_name] = [] 
    
    for time_vals, amplitude_vals in waves:
        time_vals = np.array(time_vals)
        amplitude_vals = np.array(amplitude_vals)

        dt = np.mean(np.diff(time_vals))
        fft_vals = np.fft.fft(amplitude_vals)
        freqs = np.fft.fftfreq(len(amplitude_vals), dt)

        filter_mask = np.abs(freqs) < cutoff
        fft_filtered = fft_vals * filter_mask

        amplitude_filtered = np.fft.ifft(fft_filtered).real
        
        filtered_waveforms[folder_name].append((time_vals, amplitude_filtered))


In [ ]:
plt.figure(figsize=(10, 6), dpi=150)

colors = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'cyan'] 
for i, (folder_name, waveforms) in enumerate(filtered_waveforms.items()):
    if count >= 3:
        break
    results = []
    
    for time_arr, val_arr in waveforms:
        min_val = np.min(val_arr)   # minimum amplitude value of filtered waveform
        results.append(min_val)
        
    plt.hist(results, bins=50, alpha=0.5, density=True, color=colors[i % len(colors)],
             label=folder_name, edgecolor='black')

plt.xlabel('Minimum FFT Magnitude')
plt.ylabel('Probability Density')
plt.title('Normalized Histogram of Minimum Magnitude Values - All Folders')
plt.legend(fontsize='small')
plt.grid(True)
plt.show()

In [ ]:
colors = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'cyan']

plt.figure(figsize=(10, 6), dpi=150)

for i, (folder_name, waveforms) in enumerate(filtered_waveforms.items()):
    times_of_arrival = []

    for time_vals, amplitude_vals in waveforms:
        time_vals = np.array(time_vals) - time_vals[0]
        amplitude_vals = np.array(amplitude_vals)

        V_min = np.min(amplitude_vals)
        min_idx = np.argmin(amplitude_vals)
        threshold = V_min / 2

        crossing_idx = None
        for idx in range(min_idx - 1, 0, -1):
            if amplitude_vals[idx] > threshold and amplitude_vals[idx + 1] <= threshold:
                crossing_idx = idx
                break

        if crossing_idx is not None:
            t1, v1 = time_vals[crossing_idx], amplitude_vals[crossing_idx]
            t2, v2 = time_vals[crossing_idx + 1], amplitude_vals[crossing_idx + 1]
            slope = (v2 - v1) / (t2 - t1)
            t_cross = t1 + (threshold - v1) / slope
            times_of_arrival.append(t_cross)
        else:
            times_of_arrival.append(np.nan)

    times_of_arrival = np.array(times_of_arrival)
    valid_times = times_of_arrival[~np.isnan(times_of_arrival)]
    if len(valid_times) == 0:
        print(f"No valid times for folder {folder_name}")
        continue
        
    mu = np.mean(valid_times)
    sigma = np.std(valid_times)
    print(f"Folder: {folder_name}")
    print(f"Mean arrival time (μ): {mu:.3e} s")
    print(f"Standard deviation (σ): {sigma:.3e} s\n")

    
    plt.hist(valid_times, bins=50, alpha=0.5, density=True,
             color=colors[i % len(colors)],
             label=folder_name, edgecolor='black')

plt.xlabel('Time of Arrival (s)')
plt.ylabel('Normalized Counts (Probability Density)')
plt.title('Normalized Time of Arrival Histogram for All Folders')
plt.legend(fontsize='small')
plt.grid(True)
plt.show()

In [ ]:
colors = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'cyan']
zero_threshold = 1e-4  # threshold to treat as zero crossing, adjust if needed

plt.figure(figsize=(10, 6), dpi=150)

for i, (folder_name, waveforms) in enumerate(filtered_waveforms.items()):
    pulse_areas = []

    for time_vals, amplitude_vals in waveforms:
        time_vals = np.array(time_vals) - time_vals[0]  # normalize time start to zero
        amplitude_vals = np.array(amplitude_vals)

        # Find index of minimum amplitude (pulse minimum)
        min_idx = np.argmin(amplitude_vals)

        # Find zero crossing before min_idx (leading edge)
        leading_zero_idx = None
        for j in range(min_idx, 0, -1):
            if amplitude_vals[j] * amplitude_vals[j - 1] < 0 or abs(amplitude_vals[j]) < zero_threshold:
                leading_zero_idx = j
                break

        # Find zero crossing after min_idx (trailing edge)
        trailing_zero_idx = None
        for j in range(min_idx, len(amplitude_vals) - 1):
            if amplitude_vals[j] * amplitude_vals[j + 1] < 0 or abs(amplitude_vals[j]) < zero_threshold:
                trailing_zero_idx = j
                break
                
        # If both zero crossings found, integrate pulse area
        if leading_zero_idx is not None and trailing_zero_idx is not None and trailing_zero_idx > leading_zero_idx:
            idx_range = slice(leading_zero_idx, trailing_zero_idx + 1)
            area = np.trapz(amplitude_vals[idx_range], time_vals[idx_range])
            pulse_areas.append(area)
        else:
            # If zero crossings not found, skip this waveform or handle differently
            continue

    if len(pulse_areas) == 0:
        print(f"No valid pulse areas in folder {folder_name}")
        continue

    # Plot normalized histogram for this folder
    plt.hist(pulse_areas, bins=50, alpha=0.5, density=True,
             color=colors[i % len(colors)],
             label=folder_name, edgecolor='black')

plt.xlabel('Pulse Area (Integral)')
plt.ylabel('Normalized Counts (Probability Density)')
plt.title('Normalized Pulse Area Distribution for All Folders')
plt.legend(fontsize='small')
plt.grid(True)
plt.show()


In [ ]:
colors = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'cyan']

plt.figure(figsize=(12, 7), dpi=150)

for i, (folder_name, waveforms) in enumerate(filtered_waveforms.items()):
    if len(waveforms) == 0:
        continue

    min_time = min(w[0][0] for w in waveforms)
    max_time = max(w[0][-1] for w in waveforms)

    common_time = np.linspace(min_time, max_time, 1000)

    interpolated_amps = []
    for time_vals, amp_vals in waveforms:
        time_vals = np.array(time_vals)
        amp_vals = np.array(amp_vals)
        interp_amp = np.interp(common_time, time_vals, amp_vals)
        interpolated_amps.append(interp_amp)

    avg_amp = np.mean(interpolated_amps, axis=0)

    plt.plot(common_time, avg_amp, color=colors[i % len(colors)], label=folder_name)

plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.title('Average Waveforms per Folder')
plt.grid(True)
plt.legend(fontsize='small')
plt.show()

In [ ]:
filtered_waveforms_cutoff = {}
for folder_name, waveforms in filtered_waveforms.items():
    times_of_arrival = []

    for time_vals, amplitude_vals in waveforms:
        time_vals = np.array(time_vals) - time_vals[0]
        amplitude_vals = np.array(amplitude_vals)

        V_min = np.min(amplitude_vals)
        min_idx = np.argmin(amplitude_vals)
        threshold = V_min / 2

        crossing_idx = None
        for idx in range(min_idx - 1, 0, -1):
            if amplitude_vals[idx] > threshold and amplitude_vals[idx + 1] <= threshold:
                crossing_idx = idx
                break

        if crossing_idx is not None:
            t1, v1 = time_vals[crossing_idx], amplitude_vals[crossing_idx]
            t2, v2 = time_vals[crossing_idx + 1], amplitude_vals[crossing_idx + 1]
            slope = (v2 - v1) / (t2 - t1)
            t_cross = t1 + (threshold - v1) / slope
        else:
            t_cross = np.nan
        
        times_of_arrival.append(t_cross)

    times_of_arrival = np.array(times_of_arrival)
    valid_times = times_of_arrival[~np.isnan(times_of_arrival)]

    if len(valid_times) == 0:
        print(f"No valid pulses in folder {folder_name}")
        filtered_waveforms_cutoff[folder_name] = []
        continue

    mu = np.mean(valid_times)
    sigma = np.std(valid_times)
    cutoff_time = mu + 2 * sigma

    filtered_waveforms_cutoff[folder_name] = []
    for (time_vals, amplitude_vals), t_arrival in zip(waveforms, times_of_arrival):
        if not np.isnan(t_arrival) and t_arrival <= cutoff_time:
            filtered_waveforms_cutoff[folder_name].append((time_vals, amplitude_vals))

    print(f"{folder_name}: kept {len(filtered_waveforms_cutoff[folder_name])} of {len(waveforms)} waveforms after cutoff")

In [ ]:
print(filtered_waveforms_cutoff)

In [ ]:
plt.figure(figsize=(10, 6), dpi=150)

colors = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'cyan'] 
for i, (folder_name, waveforms) in enumerate(filtered_waveforms_cutoff.items()):
    results = []
    
    for time_arr, val_arr in waveforms:
        min_val = np.min(val_arr)   
        results.append(min_val)
        
    plt.hist(results, bins=50, alpha=0.5, density=True, color=colors[i % len(colors)],
             label=folder_name, edgecolor='black')

plt.xlabel('Minimum FFT Magnitude')
plt.ylabel('Probability Density')
plt.title('Normalized Histogram of Minimum Magnitude Values - All Folders')
plt.legend(fontsize='small')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6), dpi=150)

colors = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'cyan'] 
for i, (folder_name, waveforms) in enumerate(filtered_waveforms_cutoff.items()):
    results = []
    
    for time_arr, val_arr in waveforms:
        min_val = np.min(val_arr)   # minimum amplitude value of filtered waveform
        results.append(min_val)
        
    plt.hist(results, bins=50, alpha=0.5, density=True, color=colors[i % len(colors)],
             label=folder_name, edgecolor='black')

plt.xlabel('Minimum FFT Magnitude')
plt.ylabel('Probability Density')
plt.title('Normalized Histogram of Minimum Magnitude Values - All Folders')
plt.legend(fontsize='small')
plt.grid(True)
plt.show()

In [ ]:
colors = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'cyan']

plt.figure(figsize=(10, 6), dpi=150)

for i, (folder_name, waveforms) in enumerate(filtered_waveforms_cutoff.items()):
    times_of_arrival = []

    for time_vals, amplitude_vals in waveforms:
        time_vals = np.array(time_vals) - time_vals[0]
        amplitude_vals = np.array(amplitude_vals)

        V_min = np.min(amplitude_vals)
        min_idx = np.argmin(amplitude_vals)
        threshold = V_min / 2

        crossing_idx = None
        for idx in range(min_idx - 1, 0, -1):
            if amplitude_vals[idx] > threshold and amplitude_vals[idx + 1] <= threshold:
                crossing_idx = idx
                break

        if crossing_idx is not None:
            t1, v1 = time_vals[crossing_idx], amplitude_vals[crossing_idx]
            t2, v2 = time_vals[crossing_idx + 1], amplitude_vals[crossing_idx + 1]
            slope = (v2 - v1) / (t2 - t1)
            t_cross = t1 + (threshold - v1) / slope
            times_of_arrival.append(t_cross)
        else:
            times_of_arrival.append(np.nan)

    times_of_arrival = np.array(times_of_arrival)
    valid_times = times_of_arrival[~np.isnan(times_of_arrival)]
    if len(valid_times) == 0:
        print(f"No valid times for folder {folder_name}")
        continue

    plt.hist(valid_times, bins=50, alpha=0.5, density=True,
                 color=colors[i % len(colors)],
                 label=folder_name, edgecolor='black')

plt.xlabel('Time of Arrival (s)')
plt.ylabel('Normalized Counts (Probability Density)')
plt.title('Normalized Time of Arrival Histogram for All Folders')
plt.legend(fontsize='small')
plt.grid(True)
plt.show()

In [ ]:
colors = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'cyan']
zero_threshold = 1e-4  # threshold to treat as zero crossing, adjust if needed

plt.figure(figsize=(10, 6), dpi=150)

for i, (folder_name, waveforms) in enumerate(filtered_waveforms.items()):
    pulse_areas = []

    for time_vals, amplitude_vals in waveforms:
        time_vals = np.array(time_vals) - time_vals[0]  # normalize time start to zero
        amplitude_vals = np.array(amplitude_vals)

        # Find index of minimum amplitude (pulse minimum)
        min_idx = np.argmin(amplitude_vals)

        # Find zero crossing before min_idx (leading edge)
        leading_zero_idx = None
        for j in range(min_idx, 0, -1):
            if amplitude_vals[j] * amplitude_vals[j - 1] < 0 or abs(amplitude_vals[j]) < zero_threshold:
                leading_zero_idx = j
                break

        # Find zero crossing after min_idx (trailing edge)
        trailing_zero_idx = None
        for j in range(min_idx, len(amplitude_vals) - 1):
            if amplitude_vals[j] * amplitude_vals[j + 1] < 0 or abs(amplitude_vals[j]) < zero_threshold:
                trailing_zero_idx = j
                break
                
        # If both zero crossings found, integrate pulse area
        if leading_zero_idx is not None and trailing_zero_idx is not None and trailing_zero_idx > leading_zero_idx:
            idx_range = slice(leading_zero_idx, trailing_zero_idx + 1)
            area = np.trapz(amplitude_vals[idx_range], time_vals[idx_range])
            pulse_areas.append(area)
        else:
            # If zero crossings not found, skip this waveform or handle differently
            continue

        if len(pulse_areas) == 0:
            print(f"No valid pulse areas in folder {folder_name}")
            continue

        # Plot normalized histogram for this folder
        plt.hist(pulse_areas, bins=50, alpha=0.5, density=True,
                 color=colors[i % len(colors)],
                 label=folder_name, edgecolor='black')

plt.xlabel('Pulse Area (Integral)')
plt.ylabel('Normalized Counts (Probability Density)')
plt.title('Normalized Pulse Area Distribution for All Folders')
plt.legend(fontsize='small')
plt.grid(True)
plt.show()


In [ ]:
colors = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'cyan']

plt.figure(figsize=(12, 7), dpi=150)

for i, (folder_name, waveforms) in enumerate(filtered_waveforms_cutoff.items()):
    if len(waveforms) == 0:
        continue

    min_time = min(w[0][0] for w in waveforms)
    max_time = max(w[0][-1] for w in waveforms)

    common_time = np.linspace(min_time, max_time, 1000)

    interpolated_amps = []
    for time_vals, amp_vals in waveforms:
        time_vals = np.array(time_vals)
        amp_vals = np.array(amp_vals)
        interp_amp = np.interp(common_time, time_vals, amp_vals)
        interpolated_amps.append(interp_amp)

    avg_amp = np.mean(interpolated_amps, axis=0)

    plt.plot(common_time, avg_amp, color=colors[i % len(colors)], label=folder_name)

plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.title('Average Waveforms per Folder')
plt.grid(True)
plt.legend(fontsize='small')
plt.show()